# Feature Engineering

### Importar librerias

In [1]:
import pandas as pd
import dask.dataframe as dd
from datetime import timedelta
import numpy as np

### Leer la data

In [2]:
route = 'processing_data_0001_dummie.parquet'
df = dd.read_parquet(route)

### Features

*  **Total de transacciones acumuladas en la ventana de 24 horas**: Esta columna va a llevar el conteo desde un mismo flujo de moneda( account -> subsidiary) en una ventana de 24 horas de las transacciones. El proposito de esta columna sera calcular la frecuencia transaccional en cada flujo de moneda durante la ventana de tiempo

In [3]:
def calculate_total_transactions(partition):
    # Ordenar el dataframe por account_number y transaction_date
    partition = partition.sort_values(['account_number', 'transaction_date'])

    # Calcular la diferencia en tiempo entre transacciones dentro de un mismo account_number y subsidiary
    time_difference = partition.groupby(['account_number', 'subsidiary'])['transaction_date'].diff()

    # Filtrar solo las transacciones dentro de 24 horas
    within_24_hours = time_difference <= timedelta(days=1)

    # Calcular el total de transacciones dentro de 24 horas y crear una nueva columna 'total_transactions'
    partition['total_transactions'] = within_24_hours.groupby([partition['account_number'], partition['subsidiary']]).cumsum() + 1

    return partition

*  **Total en moneda acumulada en la ventana de 24 horas:** Esta columna va a llevar una suma acumulada de la cantidad de moneda (transaction_amount) objeto de la transacción en la ventana de tiempo. El proposito de esta columna sera calcular el movimiento de moneda en cada flujo. 

In [4]:
def calculate_total_transaction_amount(partition):
    # Ordenar el dataframe por account_number y transaction_date
    partition = partition.sort_values(['account_number', 'transaction_date'])

    # Calcular la diferencia en tiempo entre transacciones dentro de un mismo account_number y subsidiary
    time_difference = partition.groupby(['account_number', 'subsidiary'])['transaction_date'].diff()

    # Filtrar solo las transacciones dentro de 24 horas
    within_24_hours = time_difference <= timedelta(days=1)

    # Calcular el total del transaction_amount dentro de 24 horas y crear una nueva columna 'total_transaction_amount'
    partition['total_transaction_amount'] = within_24_hours * partition['transaction_amount']
    partition['total_transaction_amount'] = partition.groupby([partition['account_number'], partition['subsidiary']])['total_transaction_amount'].cumsum() + partition['transaction_amount']

    return partition

* **Diferencia de tiempo en minutos entre cada transaccion:** Esta columna resta el tiempo de la ultima transaccion con la inmediatamente anterior y nos da un tiempo en minutos. El proposito de esta columna es identificar los tiempos entre transacciones en esa ventana de 24 horas

In [5]:
def calculate_time_diff_minutes(partition):
    # Ordenar el dataframe por account_number y transaction_date
    partition = partition.sort_values(['account_number', 'transaction_date'])

    # Calcular la diferencia en tiempo entre transacciones dentro de un mismo account_number y subsidiary
    time_difference = partition.groupby(['account_number', 'subsidiary'])['transaction_date'].diff()

    # Filtrar solo las transacciones dentro de 24 horas
    within_24_hours = time_difference <= timedelta(days=1)

    # Calcular la diferencia de tiempo en minutos y crear una nueva columna 'time_diff_minutes'
    partition['time_diff_minutes'] = time_difference.dt.total_seconds() / 60
    partition['time_diff_minutes'] = partition['time_diff_minutes'].where(within_24_hours, 0)

    return partition

*  **Promedio del tiempo en minutos de las transacciones para el flujo de moneda**: Esta columa suma los tiempos entre transacciones para cada flujo de moneda y lo divide por el total de transacciones acumulado en ese registro. El proposito de esta columna es llevar un rate del tiempo entre transacciones anteriores para cada registro

In [6]:
def calculate_avg_time_min_between_transactions(partition):
    # Ordenar el dataframe por account_number y transaction_date
    partition = partition.sort_values(['account_number', 'transaction_date'])

    # Calcular el promedio del tiempo entre transacciones en cada ventana de tiempo (account_number y subsidiary)
    partition['avg_time_min_between_transactions'] = partition.groupby(['account_number', 'subsidiary'])['time_diff_minutes'].cumsum() / (partition['total_transactions'])

    return partition

In [7]:
# Aplicar cada una de las funciones a cada partición del dataframe
df = df.map_partitions(calculate_total_transactions)
df = df.map_partitions(calculate_total_transaction_amount)
df = df.map_partitions(calculate_time_diff_minutes)
df = df.map_partitions(calculate_avg_time_min_between_transactions)

* **Promedio de moneda en las transacciones durante la ventana de 24 horas:** Esta columna divide el acumulado de moneda para cada flujo de moneda por el total de transacciones dado el registro. El proposito de esta columna es determinar el rate de moneda para cada transaccion segun el historico del flujo de moneda

In [8]:
df['avg_amount_transactions'] = df['total_transaction_amount'] / df['total_transactions']

In [9]:
# Convertir el dataframe de Dask a pandas para visualizar los resultados
df_featured = df.compute()

In [10]:
df_featured

,merchant_id,_id,subsidiary,transaction_date,account_number,user_id,transaction_amount,transaction_type,date,total_transactions,total_transaction_amount,time_diff_minutes,avg_time_min_between_transactions,avg_amount_transactions
1617213,817d18cd3c31e40e9bff0566baae7758,71285b05e5cdb6c1f91db657156f164b,18ea9f9ee2e7bb315f867d8e480c7f7e,2021-09-11 14:02:12,000009a84c209a6a3a97aa0d8d3c5241,47f07ac4ded748b5c29f41bc38dad56e,59.445,DEBITO,2021-09-11,1,59.445,0.0,0.0,59.445
1617214,817d18cd3c31e40e9bff0566baae7758,62bde78b842a74c0db8181896f4eb719,4f4ab38e16c10b66f82476d6c830f994,2021-09-20 16:57:28,000009a84c209a6a3a97aa0d8d3c5241,47f07ac4ded748b5c29f41bc38dad56e,178.334,DEBITO,2021-09-20,1,178.334,0.0,0.0,178.334
903308,817d18cd3c31e40e9bff0566baae7758,eb8af4d1d3aaa064282bfa29f2693627,4b4afd0fd1befe64376311475f67839c,2021-11-28 11:10:03,00000f81a1d5583a06a5cb7f4a1cbc0e,074649d5eba4aa08426876514a9b3a39,59.445,DEBITO,2021-11-28,1,59.445,0.0,0.0,59.445
942239,075d178871d8d48502bf1f54887e52fe,a6c3f05053b62c20b8fbf3fdbab5474e,ff3938bc8fcbe4b181a01472eef52ec5,2021-02-20 16:20:34,000011b29a455d40b5aea0cfca1aa7f6,a72a851ca890c7446682703de095a911,23.778,DEBITO,2021-02-20,1,23.778,0.0,0.0,23.778
12734,075d178871d8d48502bf1f54887e52fe,b9c961e11aae7ec9ec4db33487be0aa8,40d92864e9f2bfb038a41804045cd700,2021-04-28 08:26:52,000011b29a455d40b5aea0cfca1aa7f6,a72a851ca890c7446682703de095a911,11.889,DEBITO,2021-04-28,1,11.889,0.0,0.0,11.889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3974,075d178871d8d48502bf1f54887e52fe,03cd464f7df8bb4d888e2499e1a4f075,1dcc6a28fe0d679b46762077c99b96a5,2021-07-02 11:52:14,fffffe2a4fc846166ad89e0c2667733c,97895ad010f9a04d0b09e622f1cc9d0e,47.556,DEBITO,2021-07-02,1,47.556,0.0,0.0,47.556
346223,075d178871d8d48502bf1f54887e52fe,2b6679074f238d7ca01d68c2eaa6b11a,1dcc6a28fe0d679b46762077c99b96a5,2021-07-14 16:50:41,fffffe2a4fc846166ad89e0c2667733c,97895ad010f9a04d0b09e622f1cc9d0e,47.556,DEBITO,2021-07-14,1,47.556,0.0,0.0,47.556
171557,075d178871d8d48502bf1f54887e52fe,48537106f320910f8d393647566f0da3,1dcc6a28fe0d679b46762077c99b96a5,2021-07-21 15:47:00,fffffe2a4fc846166ad89e0c2667733c,97895ad010f9a04d0b09e622f1cc9d0e,61.822,DEBITO,2021-07-21,1,61.822,0.0,0.0,61.822
106905,817d18cd3c31e40e9bff0566baae7758,8e39377297f411cc4e783144a9ec2196,dc082cf1a4eb0ad0637fe0d2b986bee4,2021-11-18 18:39:03,fffffe2a4fc846166ad89e0c2667733c,97895ad010f9a04d0b09e622f1cc9d0e,47.556,DEBITO,2021-11-18,1,47.556,0.0,0.0,47.556


# Output feature Engineering

Decido guardar el archivo .parquet con el feauture engineering para usarlo en el futuro sin necesidad de correr todo el proceso dada las limitaciones de computo. 

In [11]:
# Guardar el DataFrame procesado en formato Parquet
df_featured.to_parquet('featured_data_0001.parquet')